In [ ]:
import json
import pandas as pd
import csv
import requests

with open('faculty.json', 'r') as file:
    json_data = file.read()

# Parse JSON data
data = json.loads(json_data)

In [ ]:
# Check to see if the same faculty member works in different schools
faculty_map = {}
cnt = 0
for element in data:
    person = element['name']
    institution = element['affiliation']['id']
    # print(str(person)+ " " + str(institution))
    flag = True
    if person in faculty_map:
        if faculty_map[person] == institution:
            flag = False
    if flag:
        faculty_map[person] = institution
    else:
        cnt = cnt + 1
print(cnt)

In [ ]:
# Save faculty and institution information
faculty_map = {}
# clean the names
for element in data:
    person = element['name'].strip().replace(", Ph.D.", "").replace(", M.S.", "").replace(", PhD", "").replace("Dr. ", "").replace("Mr. ", "").replace('&', ' ')
    institution = element['affiliation']['name']
    faculty_map[person] = institution

csv_file_path = 'faculty.csv'
with open(csv_file_path, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    for key, value in faculty_map.items():
        writer.writerow([key, value])

print("faculty.csv has been created.")

In [ ]:
# Process institution information
df_institution = pd.DataFrame(columns=['id', 'name', 'photoUrl'])

for element in data:
    institution = element['affiliation']
    new_row = {'id': institution['id'], 'name': institution['name'], 'photoUrl': institution['photoUrl']}
    if new_row['id'] not in df_institution['id'].values:
        df_institution = pd.concat([df_institution, pd.DataFrame([new_row])], ignore_index=True)

# df_institution.to_csv('institution.csv', index=False)
# df_institution

In [ ]:
# Standardize the Institution name to OpemAlex format
institution_url = 'https://api.openalex.org/institutions?search={}'
institution_map = {}
output_file = 'institution_map.txt'

with open(output_file, 'w') as file:
    for index, row in df_institution.iterrows():
        name = row['name']
        file.write("search " + str(index) +": " + name+ "\n")
        url = institution_url.format(name)
        page_with_results = requests.get(url).json()

        # store standard institution names and id
        results = page_with_results['results']
        # If the name of an organization is the same in the json file as it is in OpenAlex
        if len(results) > 0 and name == results[0]['display_name']:
            file.write(name+ "\n")
            file.write("id: " + results[0]['id'] + "\n")
            # institution_map[name] = name
            institution_map[name] = []
            institution_map[name].append(results[0]['id'])
            file.write("\n")
            continue
        file.write("======\n")
        # If the name is not exactly the same
        for i,res in enumerate(results):
            res_institution = res['display_name']
            institution_map[name] = []
            if name == 'University of Michigan':
                institution_map[name].append(res['id'])
                file.write(res_institution + ' ' + res['id'] + "\n")
            elif i == 0:
                institution_map[name].append(res['id'])
                file.write(res_institution  + ' ' + res['id'] + "\n")
        file.write("\n")
print(len(institution_map))

In [ ]:
# Manually add missing data
institution_map['College of William Mary'] = ['https://openalex.org/I16285277']
institution_map['University of illinois at Urbana Champaign'] = ['https://openalex.org/I157725225']
institution_map['University at Buffalo--SUNY'] = ['https://openalex.org/I63190737']
institution_map['Stony Brook University--SUNY'] = ['https://openalex.org/I59553526']
institution_map['University of Minnesota--Twin Cities'] = ['https://openalex.org/I130238516']
institution_map['University of Minnesota--Provo'] = ['https://openalex.org/I130238516']
institution_map['Pennsylvania State University--University Park'] = ['https://openalex.org/I130769515']
institution_map['Binghamton University--SUNY'] = ['https://openalex.org/I123946342']
institution_map['Rutgers University--New Brunswick'] = ['https://openalex.org/I102322142','https://openalex.org/I195342982']
institution_map['University of Pittsburgh--Pittsburgh Campus'] = ['https://openalex.org/I170201317']
institution_map['Brigham Young University--Provo'] = ['https://openalex.org/I100005738','https://openalex.org/I43943889','https://openalex.org/I110092051']
print(len(institution_map))

In [ ]:
# Manually add missing data
# institution_map['College of William Mary'] = 'William & Mary'
# institution_map['University of illinois at Urbana Champaign'] = 'University of Illinois Urbana-Champaign'
# institution_map['University at Buffalo--SUNY'] = 'University at Buffalo, State University of New York'
# institution_map['Stony Brook University--SUNY'] = 'Stony Brook University'
# institution_map['University of Minnesota--Twin Cities'] = 'University of Minnesota'
# institution_map['University of Minnesota--Provo'] = 'University of Minnesota'
# institution_map['Pennsylvania State University--University Park'] = 'Pennsylvania State University'
# institution_map['Binghamton University--SUNY'] = 'Binghamton University'
# institution_map['Rutgers University--New Brunswick'] = 'Rutgers, The State University of New Jersey'
# institution_map['University of Pittsburgh--Pittsburgh Campus'] = 'University of Pittsburgh'
# # 特殊处理下密歇根
# institution_map['University of Michigan'] = 'University of Michigan'
# print(len(institution_map))

In [ ]:
# Process and store institution node information
institution_info = pd.DataFrame(columns=['id', 'ror', 'name','relevance_score', 'country_code'])
institution_url = 'https://api.openalex.org/institutions?search={}'
institution_id = {}

for key, value in institution_map.items():
    url = institution_url.format(value.replace('&', ' '))
    print('\n' + url)
    page_with_results = requests.get(url).json()

    results = page_with_results['results']
    if len(results) > 0:
        res = results[0]
        new_row = {'id': res['id'], 'ror': res['ror'], 'name': res['display_name'], 'relevance_score': res['relevance_score'], 'country_code': res['country_code']}
        institution_info = pd.concat([institution_info, pd.DataFrame([new_row])], ignore_index=True)
        institution_id[res['display_name']] = res['id']
institution_info.to_csv('institution_info.csv', index=False)
print("create institution_info.csv")

In [ ]:
# print(find_cnt)
# author_url_with_page = 'https://api.openalex.org/authors?search={}&select=id,display_name,last_known_institution,works_api_url&page={}'
#
# output_file = "find_info.txt"
# with open(output_file, 'w') as file:
#
#     for author, institution in faculty_map.items():
#         if author in is_find:
#             print("continue")
#             continue
#         # file.write((author+" | "+ institution+"\n")
#         # string = author + " | " + institution + "\n"
#         # encoded_string = string.encode('utf-8')  # 转换为utf-8编码
#         # file.write(encoded_string.decode('utf-8'))
#
#         print(author+" "+ institution)
#         page = 1
#         has_more_pages = True
#         not_stop = True
#
#         # loop through pages
#         while has_more_pages and not_stop:
#
#             # set page value and request page from OpenAlex
#             url = author_url_with_page.format(author,page)
#             print(url)
#             # file.write(url+"\n")
#             # string = url+"\n"
#             # encoded_string = string.encode('utf-8')  # 转换为utf-8编码
#             # file.write(encoded_string.decode('utf-8'))
#
#             page_with_results = requests.get(url).json()
#
#             # loop through partial list of results
#             results = page_with_results['results']
#             flag = False
#             for i,res in enumerate(results):
#                 if res['last_known_institution'] is None:
#                     continue
#                 res_institution = res['last_known_institution']['display_name']
#                 # print(res_institution)
#                 # print(institution_map[institution])
#                 if res_institution.find(institution_map[institution]) != -1:
#                     new_row = {'author': author, 'institution': res['last_known_institution']['id'], 'works_url': res['works_api_url']}
#                     whole_info = pd.concat([whole_info, pd.DataFrame([new_row])], ignore_index=True)
#                     print("find!")
#                     file.write("find!"+"\n")
#                     print(res['works_api_url'])
#                     file.write(res['works_api_url']+"\n")
#                     flag = True
#                     find_cnt = find_cnt + 1
#                     is_find.append(author)
#                     break
#             if flag:
#                 break
#             # next page
#             page += 1
#
#             # end loop when either there are no more results on the requested page
#             # or the next request would exceed 10,000 results
#             per_page = page_with_results['meta']['per_page']
#             has_more_pages = len(results) == per_page
#             not_stop = page < 40
#             if ~not_stop:
#                 is_find.append(author)
#         print()
#         file.write("\n")
#     print(find_cnt)
#     file.write("find: "+find_cnt+"\n")
# whole_info

In [ ]:
total_cnt = 0
find_cnt = 0
whole_info = pd.DataFrame(columns=['author', 'institution', 'works_url'])
is_find = []
not_found = pd.DataFrame(columns=['author', 'institution'])

In [ ]:
# print(find_cnt)

# use "name" and "institution_id" to find "work_url"
author_url_with_page = 'https://api.openalex.org/authors?search={}&filter=last_known_institution.id:{}&select=id,display_name,last_known_institution,works_api_url&per-page=200'
output_file = 'faculty_find_result'

with open(output_file, 'a', encoding='utf-8') as file:
    for author, institution in faculty_map.items():
        # if the author has been searched
        if author in is_find:
            print("continue")
            continue
        flag = False

        print(author+ ' | ' + institution)
        for in_id in institution_map[institution]:
            # set page value and request page from OpenAlex
            url = author_url_with_page.format(author,in_id)
            print(url)
            total_cnt = total_cnt + 1

            page_with_results = requests.get(url).json()
            # loop through partial list of results
            results = page_with_results['results']
            is_find.append(author)
            if len(results) > 0:
                print(author +' | ' +  in_id + ' | ' + str(len(results)))
                file.write( author +' | ' +  in_id + ' | ' + str(len(results))+'\n' )
                for res in results:
                    if res['last_known_institution'] is not None:
                        new_row = {'author': author, 'institution': res['last_known_institution']['id'], 'works_url': res['works_api_url']}
                        whole_info = pd.concat([whole_info, pd.DataFrame([new_row])], ignore_index=True)
                        print("find!")
                        print(res['works_api_url'])
                        find_cnt = find_cnt + 1
                        flag = True
                        break
        if ~flag:
            file.write('Not found: ' + author)
            new_row = {'author': author, 'institution': institution}
            not_found = pd.concat([not_found, pd.DataFrame([new_row])], ignore_index=True)
        print()
        file.write("\n")
    print(find_cnt)
    file.write("find: "+str(find_cnt)+"\n")
whole_info

In [ ]:
not_found

In [ ]:
whole_info

In [ ]:
whole_info.to_csv('whole_info.csv', index=False)
print(total_cnt)
print(find_cnt)

In [ ]:
# use "work_url" to find the certain publication
works_url_with_page = '{}&select=id,doi,title,publication_date&page={}&per-page=200'
publication_set = set()

# use to crate relation between "faculty" and "publication" node in Neo4j
author_works = {}
# use to crate relation between "faculty" and "institution" node in Neo4j
author_institution = pd.DataFrame(columns=['author_id', 'institution_id'])

# information of "publication" node in Neo4j
publication_info = pd.DataFrame(columns=['id', 'doi', 'title', 'publication_date'])
# information of "faculty" node in Neo4j
author_info = pd.DataFrame(columns=['id', 'name', 'works_count'])

output_file = "find_work_info.txt"
with open(output_file, 'w') as file:
    for index, row in whole_info.iterrows():
        works_url = row['works_url']
        author = works_url.split(':')[-1]

        new_row = {'author_id': works_url.split(':')[-1], 'institution_id': row['institution']}
        # store relation between "faculty" and "institution" node
        author_institution = pd.concat([author_institution, pd.DataFrame([new_row])], ignore_index=True)

        file.write(author+"\n"+ works_url+"\n")
        print(author+"\n"+ works_url)
        page = 1
        has_more_pages = True
        work_count = 0
        # loop through pages
        while has_more_pages:

            # set page value and request page from OpenAlex
            url = works_url_with_page.format(works_url,page)
            print(url)
            file.write(url+"\n")
            page_with_results = requests.get(url).json()
            work_count = page_with_results['meta']['count']

            # loop through partial list of results
            results = page_with_results['results']
            if len(results) > 0:
                author_works[author] = []
            # flag = False
            for i,res in enumerate(results):
                my_id = res['id']
                # store relation between "faculty" and "publication" node in Neo4j
                author_works[author].append(my_id)
                if my_id not in publication_set:
                    publication_set.add(my_id)
                    new_row = {'id': res['id'], 'doi': res['doi'], 'title': res['title'], 'publication_date': res['publication_date']}
                    # store publication info
                    publication_info = pd.concat([publication_info, pd.DataFrame([new_row])], ignore_index=True)

            # next page
            page += 1
            per_page = page_with_results['meta']['per_page']
            has_more_pages = len(results) == per_page
        print()
        file.write("\n")
        new_row = {'id': author, 'name': row['author'], 'works_count': work_count}
        author_info = pd.concat([author_info, pd.DataFrame([new_row])], ignore_index=True)

publication_info.to_csv('publication_info.csv', index=False)
author_info.to_csv('author_info.csv', index=False)
author_institution.to_csv('author_institution.csv', index=False)

In [ ]:
# load into csv file
csv_file_path = 'author_works.csv'
with open(csv_file_path, 'w', newline='') as file:
    writer = csv.writer(file)
    for key, value in author_works.items():
        for work in value:
            writer.writerow([key, work])

print("create：", csv_file_path)

In [ ]:
# used in Neo4j 
load csv from 'file:/// author_info.csv' as line
create(:author{id:line[0],name:line[1], works_count:line[2]})

load csv from 'file:/// institution_info.csv' as line
create(:institution {id:line[0],ror:line[1], name:line[2], relevance_score: line[3],  country_code: line[4]})

load csv from 'file:/// publication_info.csv' as line
create(:publication {id:line[0],doi:line[1], title:line[2], publication_date: line[4]})



load csv from 'file:/// author_institution.csv' as line
match(a: author {id:line[0]}) match(b: institution {id:line[1]}) 
create(a)-[r:work_in]->(b) return r;

load csv from 'file:/// author_works.csv' as line
match(a: author {id:line[0]}) match(b: publication {id:line[1]}) 
create(a)-[r: publish]->(b) return r;
